In [1]:
import numpy as np 
from itertools import groupby
import itertools
from operator import itemgetter
import os
import pyBigWig

First, we need to define the location of the fasta file, the gap file, the directory with the targets, and the directories, where to save the input and target memory maps. 

In [2]:
datasets_dir = '/data/users/goodarzilab/darya/work/Datasets'
# target_dir = '/data/users/goodarzilab/darya/work/Datasets/corderoLabData/bwOS.PDX.ATAC'
target_dir = '/data/users/goodarzilab/darya/work/Datasets'

memmap_data_dir = os.path.join(os.getcwd(), 'hg_19_mmaps_upd')
targets_memmap_data_dir = os.path.join(os.getcwd(), 'targets_memmaps_test')


fasta_file = os.path.join(datasets_dir, 'hg19.ml.fa')
gaps_file = os.path.join(datasets_dir, 'hg19_gaps.txt')

Then, we define the list with full directories of each of the targets. 

In [3]:
targets_list = os.listdir(target_dir)
targets_path_list = [os.path.join(target_dir, tar) for tar in targets_list]

Then, we need to create a dictionary with nucleotides - both lower and upper case. 

In [4]:
nucs = np.array(["A", "T", "C", "G", "N"])
mapping = {u:i for i,u in enumerate(nucs)}
nucs_lower = [nuc.lower() for nuc in nucs]
mapping_lower = {u:i for i,u in enumerate(nucs_lower)}

mapping.update(mapping_lower)
nucs = np.append(nucs, nucs_lower)
nucs

array(['A', 'T', 'C', 'G', 'N', 'a', 't', 'c', 'g', 'n'], dtype='<U1')

Next, we open the gaps file. 

In [5]:
lines = []
with open(gaps_file) as f:
    for line in f:
        lines.append(line.split())

For the sake of having a full list of chromosome names, we pul it from a random bw file. 

In [9]:
# target = pyBigWig.open(os.path.join(target_dir, 'PDX008_1.bw'), 'r')
target = pyBigWig.open(os.path.join(target_dir, 'CNhs12843.bw'), 'r')

chrom_seq = target.chroms() 
chrom_seq

{'chr1': 249250621,
 'chr10': 135534747,
 'chr11': 135006516,
 'chr12': 133851895,
 'chr13': 115169878,
 'chr14': 107349540,
 'chr15': 102531392,
 'chr16': 90354753,
 'chr17': 81195210,
 'chr18': 78077248,
 'chr19': 59128983,
 'chr2': 243199373,
 'chr20': 63025520,
 'chr21': 48129895,
 'chr22': 51304566,
 'chr3': 198022430,
 'chr4': 191154276,
 'chr5': 180915260,
 'chr6': 171115067,
 'chr7': 159138663,
 'chr8': 146364022,
 'chr9': 141213431,
 'chrX': 155270560}

With the gap information derived above, we create a dictionary where each cromosome name corresponds to the "non-gap" locations, i.e., with the gaps extracted. 
Different .bed files may be differently organized, so you need to manually define the following variables: chrom_position , gaps_start, gaps_end that correspond to the location of the chromosome name and the boundaries of the gap. 
See example below. 
Note that you need to have chrom_seq values match the values from both inputs and targets.

In [10]:
lines[1]

['0',
 'chr1',
 '124535434',
 '142535434',
 '1271',
 'N',
 '18000000',
 'heterochromatin',
 'no']

In [11]:
chrom_position = 1
gaps_start = 2
gaps_end = 3

In [12]:
chrom_gaps = dict()
for i, g in itertools.groupby(lines, lambda x: x[chrom_position]):
    if i in list(chrom_seq.keys()):
        arr =  [np.array(el[gaps_start:gaps_end + 1]).astype(int) for el in g]
        arr = sorted(arr, key=itemgetter(0))
        arr_corrected = [[int(arr[i][1]), int(arr[i+1][0])] for i in range(len(arr)-1) if int(arr[i][1]) != int(arr[i+1][0])]
        if int(arr[-1][1]) != chrom_seq[i]:
            arr_corrected.append([int(arr[-1][1]), chrom_seq[i]])        
        chrom_gaps[i] = arr_corrected

The iterator below opens a fast file, and for each header (chromosome), extracts the fill sequence as a long string. 

In [13]:
def fasta_iter(fasta_name):
    """
    modified from Brent Pedersen
    Correct Way To Parse A Fasta File In Python
    given a fasta file. yield tuples of header, sequence
    """
    fh = open(fasta_name)
    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        headerStr = header.__next__()[1:].strip()
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

The loop below uses the iterator defined above, opens the fasta file, extracts the gaps, and saves the result as a memory map. 

In [ ]:
fiter = fasta_iter(fasta_file)
for ff in fiter:
    headerStr, seq = ff
    seq = np.array(list(seq))
    print (headerStr)
    for key in chrom_gaps.keys():
        if key == headerStr:
            print (key)
            memmap_name = key + "_sequence.dta"
            els_corrected = chrom_gaps[key]
#             lengths.insert(0, 0)
            lengths = [e[1] - e[0] for e in els_corrected]
            contigs_shape = sum(lengths)
            lengths.insert(0, 0)
            lengths_idx = np.cumsum(lengths)
            chr_contigs =  np.memmap(os.path.join(memmap_data_dir, (key + "_contigs_sequence.dta")), mode='w+', dtype='float32', shape=(contigs_shape))
            for i in range(len(els_corrected)):
#                 print (i)
                seq_subset = seq[els_corrected[i][0]:els_corrected[i][1]]
                seq_subset_mapped = np.zeros((len(seq_subset),))
                for nuc in nucs:
                    j = np.where(seq_subset[0:len(seq_subset)] == nuc)[0]
                    seq_subset_mapped[j] = mapping[nuc]
#                 print (seq_subset_mapped.shape, lengths_idx[i+1] - lengths_idx[i])
                chr_contigs[lengths_idx[i]:lengths_idx[i+1]] = seq_subset_mapped
            chr_contigs.flush()

The loop below uses the iterator defined above, opens the targets bw files, extracts the gaps, and saves the result as a memory map, where all the gap-free targets are sequentially stacked. 

In [ ]:
targets_path_list = [os.path.join(target_dir, tar) for tar in targets_list]
for key in chrom_gaps.keys():
    print (key)
    memmap_name = key + "_sequence.dta"
    els_corrected = chrom_gaps[key]
    lengths = [e[1] - e[0] for e in els_corrected]
    contigs_shape = sum(lengths)
    lengths.insert(0, 0)
    lengths_idx = np.cumsum(lengths)
    chr_contigs =  np.memmap(os.path.join(targets_memmap_data_dir, (key + "_targets_contigs_sequence.dta")), mode='w+', dtype='float32', shape=(len(targets_list), contigs_shape))
    for i in range(len(els_corrected)):
        for j in range(len(targets_path_list)):
            target = pyBigWig.open(targets_path_list[j], 'r')
            chr_contigs[j, lengths_idx[i]:lengths_idx[i+1]] = target.values(key, els_corrected[i][0], els_corrected[i][1])
    chr_contigs.flush()